<a href="https://colab.research.google.com/github/moonlovist/ML_DLA/blob/main/CNN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
import os
from sklearn import preprocessing

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import  BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers import MaxPool2D
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

from sklearn.metrics import roc_curve, precision_recall_curve, accuracy_score,auc

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pathinData="/content/drive/My Drive/ML_DLA"
pathout='/content/drive/My Drive/ML_DLA/model'
model_name = '/cnn_id_SN_3_NHI_20'  #name of the final model which is saved in pathout


In [4]:
import keras
print("Predicting...")
print("====================")
LOAD_MODEL=True
if LOAD_MODEL:
  cnn = keras.models.load_model(pathout+model_name+"_best.hd5")

Predicting...


In [5]:
X = fits.open(pathinData+'/files/Classification_catalog_SN_all_NHI_20_mock_4.7.01.fits')[0].data
X = np.expand_dims(X,axis=2)
Y1_id = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[1].data
Y1 = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[0].data

In [6]:
Y1_id = Y1_id[Y1==1]
X = X[Y1==1]
mu = np.median(X,axis=(1,2))
X_test = X
for i in range(0,mu.shape[0]):
  X_test[i,:,:] = X[i,:,:]/mu[i]
Y_pred_all = cnn.predict(X_test)
Y_pred_class_all=Y_pred_all*0
Y_pred_class_all[Y_pred_all>0.5]=1

In [7]:
import pandas as pd
data = pd.DataFrame({'id':Y1_id,'prediction':Y_pred_class_all.reshape(-1)})
data_mean = data.groupby('id').agg('mean')

In [8]:
data_mean.shape[0]

16605

In [10]:
data_mean

,prediction
id,
"(1160000499,)",0.996364
"(1164000610,)",1.000000
"(1164000621,)",0.000000
"(1166000592,)",1.000000
"(1168000608,)",0.745520
...,...
"(7296000517,)",0.965517
"(7299000447,)",0.996416
"(7300000584,)",0.000000


In [12]:
print('completeness:',data_mean.loc[data_mean['prediction']>0.5].shape[0]/data_mean.shape[0])

completeness: 0.9394158386028305


Accuracy

In [4]:
X = fits.open(pathinData+'/files/Classification_catalog_SN_all_NHI_20_mock_4.7.01.fits')[0].data
X = np.expand_dims(X,axis=2)
Y1_id = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[1].data
Y1 = fits.open(pathinData+'/files/Classification_id_SN_all_NHI_20_mock_4.7.01.fits')[0].data

In [ ]:
mu = np.median(X,axis=(1,2))
X_test = X
for i in range(0,mu.shape[0]):
  X_test[i,:,:] = X[i,:,:]/mu[i]
Y_pred_all = cnn.predict(X_test)
#Y_pred = Y_pred_all.reshape(-1)*0
#Y_pred[Y_pred_all.reshape(-1)>0.5]=1

In [ ]:
import pandas as pd
data = pd.DataFrame({'id':Y1_id,'prediction':Y_pred_all.reshape(-1),'true':Y1})
data_mean = data.groupby('id').agg('mean')
data_mean['prediction']>0.5 = 1
data_mean['prediction']<0.5 = 0

In [ ]:
data_mean_completeness = data_mean.loc[data_mean['true']==1]
print('completeness:',np.count_nonzero(data_mean_completeness['prediction']==data_mean_completeness['true'])/data_mean_completeness.shape[0])
print('purity:',np.count_nonzero(data_mean['prediction']==data_mean['true'])/data_mean.shape[0])